### Import Necessary Packages

In [3]:
import boto3 
import gc
import numpy as np
import sagemaker 
from sagemaker import get_execution_role
import os

### Define Bucket and Prefix for Training and Validation data 

In [4]:
bucket = sagemaker.Session().default_bucket()
prefix = 'demand-prediction'

### Obtain Necessary Parameters 

In [5]:
role = get_execution_role()
account_id = role.split(':')[4]
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
print(account_id)
print(region)

230755935769
us-east-1


### Configure Autopilot 

In [6]:
autopilot_prefix = 'demand-prediction/autopilot' ##TODO: modify this 
input_data_config = [{
      'DataSource': {
        'S3DataSource': {
          'S3DataType': 'S3Prefix',
          'S3Uri': 's3://{}/{}/input'.format(bucket,autopilot_prefix)
        }
      },
      'TargetAttributeName': 'demand'
    }
  ]

output_data_config = {
    'S3OutputPath': 's3://{}/{}/output'.format(bucket,autopilot_prefix)
  }


In [7]:
from time import gmtime, strftime, sleep
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

auto_ml_job_name = 'automl-m5predict-' + timestamp_suffix
print('AutoMLJobName: ' + auto_ml_job_name)


sm = boto3.Session().client(service_name='sagemaker',region_name=region)

sm.create_auto_ml_job(AutoMLJobName=auto_ml_job_name,
                      InputDataConfig=input_data_config,
                      OutputDataConfig=output_data_config,
                      ProblemType='Regression',
                      AutoMLJobObjective={ 'MetricName': 'MSE' },
                      RoleArn=role)


AutoMLJobName: automl-m5predict-20-02-12-52


{'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:230755935769:automl-job/automl-m5predict-20-02-12-52',
 'ResponseMetadata': {'RequestId': '38ffd075-948d-41e5-a95d-594c8fe3cfb6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '38ffd075-948d-41e5-a95d-594c8fe3cfb6',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '99',
   'date': 'Fri, 20 Nov 2020 02:12:54 GMT'},
  'RetryAttempts': 0}}

In [ ]:
print ('JobStatus - Secondary Status')
print('------------------------------')


describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
job_run_status = describe_response['AutoMLJobStatus']
    
while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    job_run_status = describe_response['AutoMLJobStatus']
    
    print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
    sleep(30)


JobStatus - Secondary Status
------------------------------
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData


In [ ]:
best_candidate = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['BestCandidate']
best_candidate_name = best_candidate['CandidateName']
print(best_candidate)
print('\n')
print("CandidateName: " + best_candidate_name)
print("FinalAutoMLJobObjectiveMetricName: " + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
print("FinalAutoMLJobObjectiveMetricValue: " + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))
